# KBMOD Reference  
  
This notebook demonstrates a gpu-accelerated image processing framework designed for image stack and time domain analysis, compatible with FITS and numpy.

An example of the C++ interface can be found in search/src/kbmod.cpp

# Setup
Before importing, make sure to run `source setup.bash` in the root directory.  
Also be sure you are running with python3.

If you are running into trouble with importing `kbmod` and related libraries and get a `ModuleNotFoundError` or `ModelImportError`, make sure that: a) your notebook is using the correct kernel and b) the pybinds directory is in the python path. In python this is something like:

```
import sys
sys.path.insert(0, 'HOMEDIR/kbmod/search/pybinds')
```

where HOMEDIR is your home directory.

In [1]:
#everything we will need for this demo
from kbmodpy import kbmod as kb
import numpy as np
import matplotlib.pyplot as plt
import os
path = "../data/demo/"


### [psf](#psf) 
2D Point Spread Function Array  
### [raw_image](#raw)
2D Image array  

### [layered_image](#layered) 
A Complete image represented as 3 raw_image layers (science, mask, variance)   

### [image_stack](#stack)  
Stack of layered_images, intended to be the same frame captured at different times

### [stack_search](#search)  
Searches an image_stack for a moving psf

### [trajectory](#traj)
Stores an object's position and motion through an image_stack



# psf
A 2D psf kernel, for convolution and adding artificial sources to images  

This simple constructor initializes a gaussian psf with a sigma of 1.0 pixels

In [2]:
p = kb.psf(1.0)

The psf can be cast into a numpy array

In [3]:
np.array(p)

array([[0.00367206, 0.01464826, 0.02320431, 0.01464826, 0.00367206],
       [0.01464826, 0.05843356, 0.09256457, 0.05843356, 0.01464826],
       [0.02320431, 0.09256457, 0.1466315 , 0.09256457, 0.02320431],
       [0.01464826, 0.05843356, 0.09256457, 0.05843356, 0.01464826],
       [0.00367206, 0.01464826, 0.02320431, 0.01464826, 0.00367206]],
      dtype=float32)

A psf can also be initialized or set from a numpy array, but the array must be square and have odd dimensions

In [4]:
arr = np.linspace(0.0, 1.0, 9).reshape(3,3)
p2 = kb.psf(arr) # initialized from array
arr = np.square(arr)
p2.set_array(arr) # set from array
np.array(p2)

array([[0.      , 0.015625, 0.0625  ],
       [0.140625, 0.25    , 0.390625],
       [0.5625  , 0.765625, 1.      ]], dtype=float32)

There are several methods that get information about its properties

In [5]:
p.get_dim() # dimension of kernel width and height
p.get_radius() # distance from center of kernel to edge
p.get_size() # total number of pixels in the kernel
p.get_sum() # total sum of all pixels in the kernel, 
            #should be close to 1.0 for a normalized kernel

0.975315511226654

<a id="layered"></a>
# layered_image
Stores the science, mask, and variance image for a single image. The "layered" means it contains all of them together.  
It can be initialized 2 ways:  
A. Load a file

In [6]:
#im = kb.layered_image(path+"example.fits", p)

B. Generate a new image from scratch

In [7]:
im = kb.layered_image("image2", 100, 100, 5.0, 25.0, 0.0, p)
# name, width, height, background_noise_sigma, variance, capture_time

Artificial objects can easily be added into a layered_image

In [8]:
im.add_object(20.0, 35.0, 2500.0)
# x, y, flux, psf

The image pixels can be retrieved as a 2D numpy array

In [9]:
pixels = im.science()
pixels

array([[-10.626595  ,  -6.433425  ,  -0.63092774, ...,   9.480564  ,
          5.7158356 ,   3.428888  ],
       [ -3.4042237 ,   5.1546435 ,  -9.349374  , ...,   4.2239113 ,
         -1.8593729 ,  -0.2675743 ],
       [  1.3247397 ,  -3.2443523 ,  -1.388663  , ...,   1.1003296 ,
         -1.6527367 ,  -5.297088  ],
       ...,
       [ -0.2647652 ,   7.6661224 ,   2.7818446 , ...,  -5.5298233 ,
         -1.4977067 ,  -3.243896  ],
       [  0.4727264 ,  -1.4867105 ,   5.2125196 , ...,  -4.342271  ,
          0.46082076,   1.0669568 ],
       [ -6.926575  ,   0.5734095 ,   1.8180655 , ...,   0.05729845,
         -4.188006  ,  -6.2619734 ]], dtype=float32)

The image can mask itself by providing a bitmask of flags (note: masked pixels are set to -9999 so they can be distinguished later from 0.0 pixles)

In [10]:
flags = ~0
flag_exceptions = [32,39]
# mask all of pixels with flags except those with specifed combiniations
im.apply_mask_flags( flags, flag_exceptions ) 

The image can be convolved with a psf kernel

In [11]:
im.convolve_psf()
# note: This function is called interally by stack_search and doesn't need to be
# used directy. It is only exposed because it happens to be a fast 
# implementation of a generally useful function

The image at any point can be saved to a file

In [12]:
#im.save_layers(path+"/out") # file will use original name

A layered_image can have its layers set from any numpy array

In [13]:
raw = kb.raw_image( np.ones_like(pixels) )

In [14]:
im.set_science( raw )
im.set_variance( raw )
im.science()

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

Get properties

In [15]:
print('Image width = %i' % im.get_width())
print('Image height = %i' % im.get_height())
print('Image time = %i' % im.get_time())
print('Image ppi = %i' % im.get_ppi()) # pixels per image, width*height

Image width = 100
Image height = 100
Image time = 0
Image ppi = 10000


<a id="stack"></a>
# image_stack
A collection of layered_images. Used to apply operations to a group of images.  

In [16]:
count = 10
imlist = [ kb.layered_image("img"+str(n), 100, 100, 10.0, 5.0, n/count, p) for n in range(count) ]
stack = kb.image_stack( imlist )
# this creates a stack with 10 50x50 images, and times ranging from 0 to 1

Manually set the times the images in the stack were taken 

In [17]:
stack.set_times( [0,2,3,4.5,5,6,7,10,11,14] )

A shortcut is provided to initialize a stack automatically from a list of files. If 'MJD' is in the header for each image, the stack will automatically load the times as well. If not, you can set them as above.

In [18]:
import os
if os.path.exists(path):
    files = os.listdir(path)
    files = [path+f for f in files if '.fits' in f]
    files.sort()
    print('Using loaded files:')
    print(files)
    stack = kb.image_stack(files)
else:
    print('Cannot find data directory. Using fake images.')

Cannot find data directory. Using fake images.


A global mask can be generated and applied to the stack

In [19]:
flags = ~0 # mask pixels with any flags
flag_exceptions = [32,39] # unless it has one of these special combinations of flags
global_flags = int('100111', 2) # mask any pixels which have any of 
# these flags in more than two images

Most features of the layered_image can be used on the whole stack

In [20]:
stack.apply_mask_flags(flags, flag_exceptions)
stack.apply_global_mask(global_flags, 2)
stack.convolve_psf()
stack.get_width()
stack.get_height()
stack.get_ppi()
stack.get_images() # retrieves list of layered_images back from the stack
stack.get_times()

[0.0, 2.0, 3.0, 4.5, 5.0, 6.0, 7.0, 10.0, 11.0, 14.0]

Here, we will create a very bright object and add it to the images and create a new image stack with the new object.

In [21]:
im_list = stack.get_images()

In [22]:
new_im_list = []
for im, time in zip(im_list, stack.get_times()):
    im.add_object(20.0+(time*8.), 35.0+(time*0.), 25000.0)
    new_im_list.append(im)

In [23]:
stack = kb.image_stack(new_im_list)

<a id="search"></a>
# stack_search
Searches a stack of images for a given psf

In [24]:
search = kb.stack_search( stack )

To save psi and images, a directory with "psi" and "phi" folders must be specified.

In [25]:
if os.path.exists(path):
    if os.path.exists(os.path.join(path,'out/psi')) is False:
        os.mkdir(os.path.join(path,'out/psi'))
    
    if os.path.exists(os.path.join(path,'out/phi')) is False:
        os.mkdir(os.path.join(path,'out/phi'))

    search.save_psi_phi(os.path.join(path, 'out'))
else:
    print('Data directory does not exist. Skipping file operations.')

Data directory does not exist. Skipping file operations.


Launch a search

In [26]:
search.search(10, 10, -0.1, 0.1, 5, 15, 2)
# angle_steps, velocity_steps, min_angle, max_angle, min_velocity, max_velocity, min_observations

Save the results to a files  
note: format is {x, y, xv, yv, likelihood, flux}

In [27]:
if os.path.exists(path):
    search.save_results(path+"results.txt", 0.05)
    # path, fraction of total results to save in file
else:
    print('Data directory does not exist. Skipping file operations.')

Data directory does not exist. Skipping file operations.


Trajectories can be retrieved directly from search without writing and reading to file.  
However, this is not recommended for a large number of trajectories, as it is not returned as a numpy array, but as a list of the trajectory objects described below

In [28]:
top_results = search.get_results(0, 100)
# start, count

<a id="traj"></a>
# trajectory
A simple container with properties representing an object and its path

In [29]:
best = top_results[0]

In [30]:
# these numbers are wild because mask flags and search parameters above were chosen randomly
best.flux 
best.lh
best.x
best.y
best.x_v
best.y_v

1.1188055276870728

tests/test_search.py shows a simple example of how to generate a set of images, add an artificial source, and recover it with search

In [31]:
# These top_results are all be duplicating searches on the same bright object we added.
top_results[:20]

[lh: 4704.088867 flux: 3326.292725 x: 52 y: 35 x_v: 13.955224 y_v: 1.118806 obs_count: 10,
 lh: 4703.314941 flux: 3325.745605 x: 52 y: 35 x_v: 13.930058 y_v: -1.397668 obs_count: 10,
 lh: 4703.314941 flux: 3325.745605 x: 52 y: 35 x_v: 13.997200 y_v: 0.279981 obs_count: 10,
 lh: 4703.314941 flux: 3325.745605 x: 52 y: 35 x_v: 14.000000 y_v: -0.000000 obs_count: 10,
 lh: 4703.314941 flux: 3325.745605 x: 52 y: 35 x_v: 13.997200 y_v: -0.279981 obs_count: 10,
 lh: 4703.314941 flux: 3325.745605 x: 52 y: 35 x_v: 13.988802 y_v: -0.559851 obs_count: 10,
 lh: 4703.314941 flux: 3325.745605 x: 52 y: 35 x_v: 13.974808 y_v: -0.839496 obs_count: 10,
 lh: 4703.314941 flux: 3325.745605 x: 52 y: 35 x_v: 13.955224 y_v: -1.118806 obs_count: 10,
 lh: 4702.495117 flux: 3325.166016 x: 52 y: 34 x_v: 13.955224 y_v: 1.118806 obs_count: 10,
 lh: 4701.964844 flux: 3324.791016 x: 52 y: 34 x_v: 13.997200 y_v: -0.279981 obs_count: 10,
 lh: 4701.964844 flux: 3324.791016 x: 52 y: 34 x_v: 13.988802 y_v: -0.559851 obs_co